In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bladder_Cancer"
cohort = "GSE185264"

# Input paths
in_trait_dir = "../../input/GEO/Bladder_Cancer"
in_cohort_dir = "../../input/GEO/Bladder_Cancer/GSE185264"

# Output paths
out_data_file = "../../output/preprocess/Bladder_Cancer/GSE185264.csv"
out_gene_data_file = "../../output/preprocess/Bladder_Cancer/gene_data/GSE185264.csv"
out_clinical_data_file = "../../output/preprocess/Bladder_Cancer/clinical_data/GSE185264.csv"
json_path = "../../output/preprocess/Bladder_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Identification of a Novel Inflamed Tumor Microenvironment Signature as a Predictive Biomarker of Bacillus Calmette-Guérin Immunotherapy in Non–Muscle-Invasive Bladder Cancer"
!Series_summary	"Improved risk stratification and predictive biomarkers of treatment response are needed for non–muscle-invasive bladder cancer (NMIBC). Here we assessed the clinical utility of targeted RNA and DNA molecular profiling in NMIBC. We performed RNA-based profiling by NanoString nCounter on non–muscle-invasive bladder cancer (NMIBC) clinical specimens and found that a novel expression signature of an inflamed tumor microenvironment (TME), but not molecular subtyping, was associated with improved recurrence-free survival after bacillus Calmette-Guérin (BCG) immunotherapy. We further demonstrated that immune checkpoint gene expression was not associated with higher recurrence rates after BCG."
!Series_overall_design	"Gene expression in NMIBC samples was profiled by 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Determine gene expression data availability
is_gene_available = True  # Based on the Series_summary and overall_design, this contains RNA-based profiling

# 2.1 Determine data availability for trait, age, and gender
# For trait (Bladder Cancer), we'll use bcg response as the trait since this is the focus of the study
trait_row = 13  # bcg.y.n field
# Age is not available in the data
age_row = None
# Gender is available
gender_row = 7  # Sex field

# 2.2 Define conversion functions for each variable
def convert_trait(value):
    """Convert BCG treatment status to binary format."""
    if value is None or pd.isna(value) or 'NA' in value:
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'Treated.BCG' in value:
        return 1  # BCG treated
    else:
        return 0  # Not treated with BCG

def convert_age(value):
    """Convert age to numeric value."""
    # Age is not available
    return None

def convert_gender(value):
    """Convert gender to binary format: female=0, male=1."""
    if value is None or pd.isna(value):
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value == 'F':
        return 0  # Female
    elif value == 'M':
        return 1  # Male
    else:
        return None  # Unknown or other

# 3. Save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available
if trait_row is not None:
    # Create the clinical data DataFrame correctly
    sample_chars = {
        0: ['studyid.sampleid: NA', 'studyid.sampleid: P-0005606-T01-IM5', 'studyid.sampleid: P-0006902-T01-IM5', 'studyid.sampleid: P-0009371-T01-IM5', 'studyid.sampleid: P-0004941-T01-IM5', 'studyid.sampleid: P-0005087-T01-IM5', 'studyid.sampleid: P-0003261-T01-IM5', 'studyid.sampleid: P-0003878-T01-IM5', 'studyid.sampleid: P-0004757-T01-IM5', 'studyid.sampleid: P-0003438-T01-IM5', 'studyid.sampleid: P-0003823-T01-IM5', 'studyid.sampleid: P-0003352-T01-IM5', 'studyid.sampleid: P-0003690-T01-IM5', 'studyid.sampleid: P-0003433-T01-IM5', 'studyid.sampleid: P-0008240-T01-IM5', 'studyid.sampleid: P-0004424-T01-IM5', 'studyid.sampleid: P-0003408-T01-IM5', 'studyid.sampleid: P-0003238-T01-IM5', 'studyid.sampleid: P-0008867-T01-IM5', 'studyid.sampleid: P-0003257-T01-IM5', 'studyid.sampleid: P-0006645-T01-IM5', 'studyid.sampleid: P-0003817-T01-IM5', 'studyid.sampleid: P-0006142-T01-IM5', 'studyid.sampleid: P-0006291-T01-IM5', 'studyid.sampleid: P-0007966-T01-IM5', 'studyid.sampleid: P-0006194-T01-IM5', 'studyid.sampleid: P-0003403-T01-IM5', 'studyid.sampleid: P-0007285-T01-IM5', 'studyid.sampleid: P-0004224-T01-IM5', 'studyid.sampleid: P-0008834-T01-IM5'],
        1: ['study: UNC', 'study: MSK'], 
        2: ['tissue: Bladder Cancer'], 
        3: ['hede: Early Basal-like (H3)', 'hede: Luminal CIS-like (H2)', 'hede: Luminal (H1)'], 
        4: ['mda: Basal', 'mda: TP53', 'mda: NA', 'mda: Luminal'], 
        5: ['lund: GenomicUnstable', 'lund: SCC-Like', 'lund: NA', 'lund: Infiltrated', 'lund: UrobasalA', 'lund: UrobasalB'], 
        6: ['immune: high', 'immune: low', 'immune: medium'], 
        7: ['Sex: F', 'Sex: M'], 
        8: ['Stage: Ta', 'Stage: T1', 'Stage: Ta/T1'], 
        9: ['grade: Low', 'grade: High', 'grade: .'], 
        10: ['cis: No', 'cis: Yes'], 
        11: ['tumor_no: NA', 'tumor_no: 1', 'tumor_no: 2'], 
        12: ['recurrence: NA', 'recurrence: No.Recurrence', 'recurrence: Recurrence'], 
        13: ['bcg.y.n: NA', 'bcg.y.n: Treated.BCG'], 
        14: ['bcg: NA', 'bcg: BCG', 'bcg: Observation', 'bcg: Cystectomy', 'bcg: MMC']
    }
    
    # Create a proper DataFrame from the sample characteristics
    clinical_data = pd.DataFrame.from_dict(sample_chars, orient='index')
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Clinical features preview:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to: {out_clinical_data_file}")


Clinical features preview:
{0: [nan, 0.0], 1: [1.0, 1.0], 2: [nan, nan], 3: [nan, nan], 4: [nan, nan], 5: [nan, nan], 6: [nan, nan], 7: [nan, nan], 8: [nan, nan], 9: [nan, nan], 10: [nan, nan], 11: [nan, nan], 12: [nan, nan], 13: [nan, nan], 14: [nan, nan], 15: [nan, nan], 16: [nan, nan], 17: [nan, nan], 18: [nan, nan], 19: [nan, nan], 20: [nan, nan], 21: [nan, nan], 22: [nan, nan], 23: [nan, nan], 24: [nan, nan], 25: [nan, nan], 26: [nan, nan], 27: [nan, nan], 28: [nan, nan], 29: [nan, nan]}
Clinical data saved to: ../../output/preprocess/Bladder_Cancer/clinical_data/GSE185264.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['53BP1', 'ABCD3', 'ACTB', 'ADIRF', 'ADPRHL2', 'AFTPH', 'AHNAK2', 'AKT',
       'ALDH1L1', 'ALOX5', 'ALOX5AP', 'ANLN', 'APEX1', 'APH1B', 'APOBEC3A',
       'APOBEC3B', 'APOBEC3C', 'APOBEC3D', 'APOBEC3F', 'APOBEC3G'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the provided gene identifiers, I can analyze whether they are standard human gene symbols or other identifiers

# Looking at the sample gene identifiers:
# - 53BP1, ACTB, AKT: These are standard human gene symbols
# - APOBEC3A, APOBEC3B, etc.: These are proper human gene symbols for the APOBEC3 family
# - ALDH1L1, ALOX5, etc.: These are standard human gene nomenclature

# All of these appear to be standard HGNC (HUGO Gene Nomenclature Committee) gene symbols
# They follow the conventional naming patterns for human genes
# No mapping appears to be needed as these are already human gene symbols

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
print(f"Original gene count: {len(gene_data)}")
print(f"Sample of gene symbols before normalization:")
print(gene_data.index[:20])  # Display first 20 gene symbols

# Normalize the gene data (skip mapping since we already have gene symbols)
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene count: {len(normalized_gene_data)}")

# Since this dataset has a small number of genes, we'll use the original data if normalization removes too many
if len(normalized_gene_data) < len(gene_data) * 0.9:  # If we lost more than 10% of genes
    print("Warning: Gene symbol normalization removed too many genes. Using original gene data without normalization.")
    normalized_gene_data = gene_data  # Use the original data without normalization

# Create directory for the gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the gene data to a CSV file
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# 2. Fix the clinical data extraction to properly use sample accessions
# Reread the clinical data from the matrix file
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
_, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Check if we have the Sample_geo_accession column to identify GSM IDs
if '!Sample_geo_accession' in clinical_data.columns:
    # Extract the GSM IDs from the clinical data
    gsm_ids = clinical_data['!Sample_geo_accession'].tolist()
    print(f"Found {len(gsm_ids)} GSM IDs in clinical data")
    print(f"First 5 GSM IDs: {gsm_ids[:5]}")
    
    # Extract clinical features with proper GSM IDs
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Transpose the clinical dataframe to have samples as rows and features as columns
    selected_clinical_df = selected_clinical_df.T
    
    # Check if the sample IDs match between clinical and gene data
    common_samples = set(selected_clinical_df.index).intersection(set(normalized_gene_data.columns))
    print(f"Number of common samples between clinical and gene data: {len(common_samples)}")
    
    # Save the properly formatted clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Create a proper linked dataset using the common samples
    if len(common_samples) > 0:
        # Filter both datasets to only include common samples
        clinical_filtered = selected_clinical_df.loc[list(common_samples)]
        gene_filtered = normalized_gene_data[list(common_samples)]
        
        # Combine the datasets
        linked_data = pd.concat([clinical_filtered, gene_filtered.T], axis=1)
        print(f"Linked data shape after proper ID matching: {linked_data.shape}")
        
        # 3. Handle missing values in the linked data with more relaxed criteria
        # First, check missing value percentages
        trait_missing = linked_data[trait].isna().mean() * 100
        print(f"Percentage of missing values in trait: {trait_missing:.2f}%")
        
        # Apply the missing value handling
        linked_data_cleaned = handle_missing_values(linked_data, trait)
        print(f"Linked data shape after handling missing values: {linked_data_cleaned.shape}")
        
        # If we still have adequate data after cleaning
        if linked_data_cleaned.shape[0] >= 5 and linked_data_cleaned.shape[1] >= 10:  # Lower threshold
            # 4. Determine whether the trait and demographic features are severely biased
            is_trait_biased, linked_data_cleaned = judge_and_remove_biased_features(linked_data_cleaned, trait)
            
            # 5. Conduct quality check and save the cohort information
            note = "Dataset contains gene expression data from bladder cancer samples with BCG treatment information."
            is_usable = validate_and_save_cohort_info(
                is_final=True, 
                cohort=cohort, 
                info_path=json_path, 
                is_gene_available=True, 
                is_trait_available=True, 
                is_biased=is_trait_biased, 
                df=linked_data_cleaned, 
                note=note
            )
            
            # 6. If the linked data is usable, save it as a CSV file
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                linked_data_cleaned.to_csv(out_data_file)
                print(f"Linked data saved to {out_data_file}")
            else:
                print(f"The dataset was determined to be not usable for analysis. Bias in trait: {is_trait_biased}")
        else:
            print("Warning: After handling missing values, insufficient data remains for analysis")
            is_usable = validate_and_save_cohort_info(
                is_final=True, 
                cohort=cohort, 
                info_path=json_path, 
                is_gene_available=True, 
                is_trait_available=True, 
                is_biased=True,
                df=linked_data_cleaned, 
                note="After cleaning, insufficient data remains for analysis."
            )
            print("The dataset was determined to be not usable for analysis.")
    else:
        print("Warning: No common samples found between clinical and gene data")
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=True,
            df=pd.DataFrame(), 
            note="No common samples found between clinical and gene data."
        )
        print("The dataset was determined to be not usable for analysis.")
else:
    print("Warning: No GSM IDs found in clinical data")
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=True,
        df=pd.DataFrame(), 
        note="No GSM IDs found in clinical data."
    )
    print("The dataset was determined to be not usable for analysis.")

Original gene count: 446
Sample of gene symbols before normalization:
Index(['53BP1', 'ABCD3', 'ACTB', 'ADIRF', 'ADPRHL2', 'AFTPH', 'AHNAK2', 'AKT',
       'ALDH1L1', 'ALOX5', 'ALOX5AP', 'ANLN', 'APEX1', 'APH1B', 'APOBEC3A',
       'APOBEC3B', 'APOBEC3C', 'APOBEC3D', 'APOBEC3F', 'APOBEC3G'],
      dtype='object', name='ID')
Normalized gene count: 432
Gene data saved to ../../output/preprocess/Bladder_Cancer/gene_data/GSE185264.csv
Found 15 GSM IDs in clinical data
First 5 GSM IDs: ['!Sample_characteristics_ch1', '!Sample_characteristics_ch1', '!Sample_characteristics_ch1', '!Sample_characteristics_ch1', '!Sample_characteristics_ch1']
Number of common samples between clinical and gene data: 78
Clinical data saved to ../../output/preprocess/Bladder_Cancer/clinical_data/GSE185264.csv
Linked data shape after proper ID matching: (78, 434)
Percentage of missing values in trait: 53.85%
Linked data shape after handling missing values: (36, 434)


Quartiles for 'Bladder_Cancer':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Bladder_Cancer' in this dataset is severely biased.

For the feature 'Gender', the least common label is '0.0' with 8 occurrences. This represents 22.22% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

The dataset was determined to be not usable for analysis. Bias in trait: True
